In [2]:
from openai import AzureOpenAI
from google.colab import userdata, drive
import pandas as pd
from tqdm import tqdm

In [3]:
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/conserta-avioes'

Mounted at /content/drive


In [4]:
api_key = userdata.get('LLM_API')

In [5]:
client = AzureOpenAI(
    api_key=api_key,
    api_version="2024-12-01-preview",
    azure_endpoint=r"https://desafio-ia-resource.openai.azure.com/"
)

In [7]:
test_messages = [
        {
            "role": "system",
            "content": "Você é um assistente útil. Responda apenas 'OK' para confirmar que a conexão está funcionando."
        },
        {
            "role": "user",
            "content": "Teste de conexão"
        }
    ]

response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=test_messages,
            temperature=1
        )

In [8]:
response

ChatCompletion(id='chatcmpl-CeZn4VunJkKHMwTfSOXvXK6YkZGTI', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='OK', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1763786734, model='gpt-5-mini-2025-08-07', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=35, total_tokens=46, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details

In [9]:
def add_context(frase):
    messages = [
            {
                "role": "system",
                "content": r"You are a data augmentation tool, specialized in the aircraft maintenance domain. When prompted with a sentence, you shall reply with 4 sentences which share the same meaning, but differ in structure and style (technical, colloquial, dry...). In your variations, make sure to mix up right\left, and numbers, when those are mentioned in the original sentence. Try to vary sentence tone and structure, but keep the wording simple."
            },
            {
                "role": "user",
                "content": "#1 ENGINE BAFFLE CRACKED."
            },
            {
                "role": "assistant",
                "content": """The baffle on the #3 engine was found cracked during inspection.
    Noticed a clear crack on the #4 engine baffle during the check.
    A crack was identified in the #2 engine baffle.
    The #3 engine has a cracked baffle."""
            },
            {
                "role": "user",
                "content": "RIGHT MAGNETOS BLAST TUBE FELL OFF."
            },
            {
                "role": "assistant",
                "content": """The blast tube for the left magneto came loose and detached.
    The right mag’s cooling tube dropped off during the flight.
    Found the blast tube on the right magneto completely separated.
    Left magneto blast tube detached."""
            },
            {
                "role": "user",
                "content": frase
            }
        ]
    return messages

In [8]:
add_context("HOLDING SHORT OF RUNWAY, AFTER COMPLETION OF RUN-UP, ENGINE WOULD NOT TAKE POWER.")

[{'role': 'system',
  'content': "You are a data augmentation tool, specialized in the aircraft maintenance domain. When prompted with a sentence, you shall reply with 4 sentences which share the same meaning, but differ in structure and style (technical, colloquial, dry...). In your variations, make sure to mix up right\\left, and numbers, when those are mentioned in the original sentence. Don't make the sentences too long, and try to vary tone and structure."},
 {'role': 'user', 'content': '#1 ENGINE BAFFLE CRACKED.'},
 {'role': 'assistant',
  'content': 'The baffle on the #3 engine was found cracked during inspection.\n    Noticed a clear crack on the #4 engine baffle during the check.\n    A crack was identified in the #2 engine baffle.\n    The #3 engine has a cracked baffle.'},
 {'role': 'user', 'content': 'RIGHT MAGNETOS BLAST TUBE FELL OFF.'},
 {'role': 'assistant',
  'content': 'The blast tube for the left magneto came loose and detached.\n    The right mag’s cooling tube drop

In [28]:
response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=add_context("HOLDING SHORT OF RUNWAY, AFTER COMPLETION OF RUN-UP, ENGINE WOULD NOT TAKE POWER."),
            temperature=1
        )
print(response.choices[0].message.content)

While holding short of the runway after completing the run-up, the engine failed to accept power.
Stopped at the hold-short line after the run-up, the engine wouldn't pick up power when advanced.
After run-up, during the hold short, the engine did not develop any power.
Holding short of the runway post run-up, the engine refused to produce power.


In [12]:
print(response.choices)

While holding short of the runway after completing the run‑up, the engine would not take power.
After run‑up and stopped at the hold short, the engine failed to accept throttle — no power increase.
Holding at the runway hold‑short line following run‑up, the engine wouldn’t accelerate when power was applied.
Run‑up complete; aircraft held short, but the engine did not respond to power inputs.


In [17]:
response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=add_context("ENGINE IDLE OVERRIDE KILLED ENGINE."),
            temperature=1
        )
print(response.choices[0].message.content)

Engaging the engine idle override shut the engine down.
The engine was killed when the idle override was activated.
Hit the idle override and the engine died.
Activation of the idle-override control resulted in engine shutdown.


In [18]:
response = client.chat.completions.create(
            model="gpt-5-mini",
            messages=add_context("#3 BRAKE MISSING SCREW."),
            temperature=1
        )
print(response.choices[0].message.content)

No. 3 brake is missing its retaining screw.
Left brake found without the screw during inspection.
Discovered #2 brake lacking a mounting screw.
Brake 3 has a missing fastener (screw).


In [29]:
response.choices[0].message.content.split('\n')

['While holding short of the runway after completing the run-up, the engine failed to accept power.',
 "Stopped at the hold-short line after the run-up, the engine wouldn't pick up power when advanced.",
 'After run-up, during the hold short, the engine did not develop any power.',
 'Holding short of the runway post run-up, the engine refused to produce power.']

In [10]:
dataset = pd.read_csv(folder+'/problemas_dedup.csv')

In [11]:
problemas = dataset['DESAB'].to_list()

In [12]:
problemas[:10]

['ENGINE IDLE OVERRIDE KILLED ENGINE.',
 'HAD ENGINE CHOKE & BRIEFLY LOSE POWER ON DEPARTURE. FULL THR',
 'HOLDING SHORT OF RUNWAY, AFTER COMPLETION OF RUN UP, ENGINE',
 '#1 ENGINE BAFFLE CRACKED.',
 'RIGHT MAGNETOS BLAST TUBE FELL OFF.',
 'ON START UP, ENGINE BEGAN TO RUN ROUGH. RAN ENGINE @ 1200 RIGHT PATTERN',
 'RIGHT ENGINE #4 AIR BAFFLE IS CRACKED',
 'RIGHT ENGINE FORWARD ALTERNATOR ATTACH BOLT LOOSE.',
 'RIGHT ENGINE, ALL VALVE COVERS ARE LEAKING.',
 'IN FLIGHT, ENGINE BECAME ROUGH. DISCOVERED RIGHT MAGNETOS INOPERABLE. E']

In [20]:
dataset_aumentado = [(i,prob,'orig') for i,prob in enumerate(problemas)]

In [14]:
pd.DataFrame(dataset_aumentado,columns=['Grupo','Frase','Tipo'])

,Grupo,Frase,Tipo
0,0,ENGINE IDLE OVERRIDE KILLED ENGINE.,orig
1,1,HAD ENGINE CHOKE & BRIEFLY LOSE POWER ON DEPAR...,orig
2,2,"HOLDING SHORT OF RUNWAY, AFTER COMPLETION OF R...",orig
3,3,#1 ENGINE BAFFLE CRACKED.,orig
4,4,RIGHT MAGNETOS BLAST TUBE FELL OFF.,orig
...,...,...,...
1623,1623,CRANKSHAFT FRONT SEAL NEEDS TO BE REPLACED.,orig
1624,1624,WORN SPOTS ON FORWARD LOWER BAFFLE SEAL ABOVE ...,orig
1625,1625,ROD CHANGE PER LYCOMING.,orig
1626,1626,#3 EXHAUST SIDE PUSH ROD & ROCKER SCORED.,orig


In [21]:
excecoes = []

In [22]:
for i,prob in tqdm(enumerate(problemas),total=len(problemas)):
  prompt = add_context(prob)
  try:
    response = client.chat.completions.create(
              model="gpt-5-mini",
              messages=prompt,
              temperature=1
          )
    variacoes = response.choices[0].message.content.split('\n')
    dataset_aumentado.extend([(i,var,'var') for var in variacoes])
  except Exception as ex:
    print("Erro!")
    print(f"Frase: {prob}")
    print(ex)
    print()
    excecoes.append((i,prob,ex))

  2%|▏         | 40/1628 [05:37<3:28:26,  7.88s/it]

Erro!
Frase: BOTH ENGS, ALL ROCKER COVERS LOOSE.
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'low'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}



  9%|▉         | 147/1628 [21:44<3:43:03,  9.04s/it]

Erro!
Frase: LEFT BAFFLE, SPARK PLUG ACCESS PLUG WORN TABS.
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}



 14%|█▍        | 230/1628 [32:24<2:51:06,  7.34s/it]

...

 95%|█████████▍| 1545/1628 [3:41:15<12:07,  8.76s/it]

Erro!
Frase: BAFFLE PLUG ABOVE #4 CYLINDER HAS BROKEN TABS.
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}



 98%|█████████▊| 1603/1628 [3:49:19<02:56,  7.06s/it]

Erro!
Frase: RIGHT BAFFLE SEAL, BY #2 CYLINDER, TORN.
Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': True, 'detected': True}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}}}



100%|██████████| 1628/1628 [3:52:45<00:00,  8.58s/it]


In [23]:
dataset_aumentado = pd.DataFrame(dataset_aumentado,columns=['Grupo','Frase','Tipo'])
dataset_aumentado.to_csv(folder+'/dataset_aumentado.csv',index=False)